<a href="https://colab.research.google.com/github/hamagami/anomaly-detection/blob/main/04_05_oneclassSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# one class SVM による時系列異常検知

### 必要なモジュールのimport

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM

### セグメントの切り出し関数

In [ ]:
from scipy import signal
#ヒストグラム　
def gethist(lst,range=(0,10),bins=25):
    win= signal.hann(len(lst)) 
    hist, bins= np.histogram(win*lst, bins, range)
    return hist, bins

In [ ]:
def segdata(lst, dim): #lst:１次元系列　dim:セグメントの幅
    segs = np.empty((0,dim), float)#0×dimの空配列
    hists = np.empty((0,25), float)#0×dimの空配列
    for i in range(lst.size - dim + 1):#1つづつずらしながらセグメントをつくっている。最後のセグメントの開始点は lst.size-dim
        hist,bins= gethist(lst[i:i+dim])
        
        seg=np.array(lst[i:i+dim][::-1].reshape((1,-1))) #セグメントの切り出し，時系列反転，appendのための2次ベクトル化
        hist=hist.reshape((1,-1))
        hists = np.append(hists,hist,axis=0)
        segs = np.append(segs,seg,axis=0)

    return segs, hists

In [ ]:
def getdata():
  !wget "www.dropbox.com/s/x3fmb9mxr4xkip3/qtdbsel102.txt" #ローカルにコピーしてくる
  LEN=3000  #分析区間

  SP=0         #学習用データの開始点
  AP=3000   #テスト用データの開始点　個のデータの場合 4250ポイント付近に異常がある
  data = np.loadtxt("qtdbsel102.txt",delimiter="\t")
  print("データ数:",data.shape[0],"  次元数:",data.shape[1])

  #元データは3次元の時系列，3次のデータ(indexとしては2)を指定して学習/テストデータに分割
  train_org = data[SP:SP+LEN, 2]      #学習用データとして 1～2999サンプル区間を使用
  test_org  = data[AP:AP+LEN, 2]    #テスト用データとして3000～5999サンプルを使用
  
  #x軸
  x=np.arange(SP,SP+LEN)

  return x, train_org, test_org

### 窓関数の設定と切り出し

In [ ]:
x, train_org, test_org = getdata()

plt.plot(x,train_org)
plt.title("train")
plt.legend()
plt.show()
plt.plot(x,test_org)
plt.title("test with anomaly")
plt.legend()
plt.show()


In [ ]:
WLEN=256#セグメントのサイズ
train_seg, train_hist= segdata(train_org, WLEN)
test_seg, test_hist = segdata(test_org, WLEN)

In [ ]:
plt.plot(train_hist.T)
plt.show()
plt.plot(test_hist.T)
plt.show()

## one class SVMによる判定



### 波形をそのまま使った場合

In [ ]:
clf = OneClassSVM(nu=0.00001, kernel='poly', gamma='auto')
clf.fit(train_seg)
pred = clf.predict(train_seg)
# train
plt.plot(train_org, label="train")
plt.title("train-wave")
plt.legend()
outlier_rows = [i for i in range(len(pred)) if pred[i]<0]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")
plt.show()
#test
plt.plot(test_org, label="test")
plt.legend()
plt.title("test-wave")
pred = clf.predict(test_seg)
outlier_rows = [i for i in range(len(pred)) if pred[i]<0]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.show()

## ヒストグラム特徴を用いた場合

In [ ]:
clf = OneClassSVM(nu=0.00001, kernel='poly', gamma='auto')
clf.fit(train_hist)
pred = clf.predict(train_hist)
#train
plt.plot(train_org)
plt.title("train-hist")
plt.legend()
outlier_rows = [i for i in range(len(pred)) if pred[i]<0]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")
plt.show()
#test
plt.plot(test_org, label="test-hist")
plt.legend()
pred = clf.predict(test_hist)
outlier_rows = [i for i in range(len(pred)) if pred[i]<0]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.show()